# DAY 2: Visium Spatial Transcriptomics Data Analysis - Mouse Intestine

This notebook will guide you through Day 2 content.
We will cover:

1. Cell type deconvolution
2. Spatial Neighbours
3. Working with Multiple Tissue Sections
4. High resolution Visium HD Data Analysis

## How to use this notebook

This notebook is intended to be used as a reference for you own analysis.
All code chunks have an explanation detailing the analysis steps and their purpose, as well as key parameters.
Play around with these and see what they do, so that you are better equiped to adapt the workflow to your own data.

## Datasets

We will be using a Visium dataset from Parigi et al, 2022.
This dataset was generated using V1 3' polyA Visium chemistry and consists of four mouse intestine samples taken from healthy mice and mice subjected to a DSS colitis model, where the intestine is damaged.
Today, we will be using healthy mouse intestine section as an example.
Overall, the dataset quality is good but there are some quality issues, which will hopefully show you what to look out for in your own data. 

https://www.nature.com/articles/s41467-022-28497-0

For HD data analysis, we will be using a public demo dataset from 10x Genomics, also from mouse intestine. The tissues are very similar, so this should give you a good sense of the differences between older and new technologies.

https://www.10xgenomics.com/datasets/visium-hd-cytassist-gene-expression-libraries-of-mouse-intestine

## 1. Cell Type Deconvolution

Many ST platforms capture gene expression data at resolutions that encompass multiple cells within each spatial spot.
In this case, each Visium spot is approximately 55 micrometres in diameter and can cover multiple cell types, particularly in dense or heterogeneous tissues.
This mixing of cell types within a single spot poses a significant challenge for understanding the cellular composition and the spatial organisation of individual cell types.

Cell type deconvolution is the process of estimating the proportions and types of different cells present within a mixed cell population, based on gene expression data.
In the context of spatial transcriptomics, deconvolution methods aim to unravel the cellular composition of each spot by inferring which cell types are present and in what proportions.
This process is crucial for interpreting the spatial organisation of cell types within tissues.

### Approaches to Cell Type Deconvolution

Several computational approaches have been developed to perform cell type deconvolution in spatial transcriptomics data. These methods generally fall into two categories:

**Reference-Based Methods:**
These methods use reference expression profiles from single-cell RNA-seq datasets to infer the cell type composition of spatial spots. They typically involve algorithms that match the expression profiles of spots to those of known cell types.

**De Novo Methods:** These methods do not rely on external reference datasets and instead use statistical models to infer transcriptional groups from the data directly.

### Considerations and Challenges

**Resolution and Spot Size:**
The resolution of the spatial platform affects the accuracy of deconvolution.
Smaller spot sizes generally yield more precise cell type compositions but may also capture fewer transcripts, leading to increased noise.

**Reference Data Quality:**
The choice and quality of the reference single-cell dataset significantly influence deconvolution accuracy.
The reference must be well-annotated and relevant to the tissue being studied.
The majority of available methods are reference-based and these tend to perform better than reference-free approaches, but this entirely hinges on using an appropriate reference.

**Reference Data Resolution:**
Many deconvolution algorithms struggle to accurately distinguish closely related cell types;
e.g. T-Cell vs B-Cell is straightforward, but CD4+ T-Cell vs CD8+ T-Cell (or even finer subsets than that) predictions should be interpreted more cautiously.

Here, we will use a reference-based method [cell2location](https://github.com/BayraktarLab/cell2location).
It is currently one of the best performing methods in Python, although many others are available.

You can read more about the cell2location method here:

https://www.nature.com/articles/s41587-021-01139-4

As a reference, we will use a mouse intestine scRNA-Seq dataset that we prepared earlier.
We can read in this data from a previously prepared [AnnData](https://anndata.readthedocs.io/en/stable/) object.

### 1.1 Set up your environment and load reference data

First we need to set up the environment and load the packages we will use for this workshop.

In [ ]:
# import os package for working with system path
import os
# import numpy for scientific computing 
import numpy as np
# import pandas for dataframe manipulation
import pandas as pd
# import matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
# import Scanpy and AnnData for single-cell RNAseq
import anndata as ad
import scanpy as sc
# import squidpy for spatial transcriptomics
import squidpy as sq
# import cell2location for cell type deconvolution
import cell2location
from cell2location.models import RegressionModel
import scvi
from scipy.stats import zscore
from scipy.sparse import csr_matrix
import bin2cell as b2c
from spatialdata_io import visium_hd
import spatialdata_plot

In the first part of today, we will be (optionally) using GPUs on the cluster to run parts of the code that would otherwise be very slow to run.
To do this from jupyter, we will use a custom package that submits chunks of code to slurm queue and loads the outputs back to the jupyter environment.
This is loaded by importing the package below. 

If you are using WIMM CCB cluster, this is also available there.
If you are using a different cluster set up, you would need to set this up outside of jupyter, or run the notebook on a node with GPU access.

In [ ]:
import ipy_slurm_exec
%load_ext ipy_slurm_exec

Set location to load data from and store analysis output.

In [ ]:
DATA_FOLDERNAME = '/nvme/project/shared/python/5_python_spatial_omics/data'
PRECOMPUTED_FOLDERNAME = '/nvme/project/shared/python/5_python_spatial_omics/data/precomputed/day1'
OUTPUT_FOLDERNAME = '/PATH/TO/YOUR/DIRECTORY'

To start with, we will load the anndata object we have worked on yesterday. A pre-computed one for the tutorial is available here if you don't want to use your own. 

In [ ]:
adata = sc.read_h5ad(
    os.path.join(PRECOMPUTED_FOLDERNAME, 'day1.h5ad')
)
adata

Quick visualisation of the spatial data that we were using yesterday.

In [ ]:
sq.pl.spatial_scatter(
    adata,
    color="clusters"
)

Cell type deconvolution requires a single cell RNA-Seq reference dataset.
Ideally, you'd have a matched reference to your experiment, although this is not always possible.
If you don't have appropriate single cell data yourself, a good place to start looking for one is [CZI cellxgene data portal](https://cellxgene.cziscience.com/datasets), which has a large assortment of single-cell datasets from all major organs that you can download as [AnnData](https://anndata.readthedocs.io/en/stable/) objects.

Here, we pre-prepared a mouse intestine single-cell reference dataset.
Let's load it and inspect it.

In [ ]:
ref = sc.read_h5ad(
    os.path.join(PRECOMPUTED_FOLDERNAME,'ref.h5ad')
)
ref

Let’s visualise the dataset - here, we can see that the dataset has all major epithelial, mesenchymal and immune cells present in the tissue.

> **Exercise:**
What would happen if your reference was incomplete/missing key cell types?
As an additional exercise, try removing cell populations from this reference to see how incomplete references impact deconvolution.

In [ ]:
sc.pl.umap(
    ref,
    color=["CellType"]
)

### 1.2 Filter data

First we want to filter out genes from the reference dataset that are likely to be uninformative. 

In [ ]:
selected = cell2location.utils.filtering.filter_genes(
    ref,
    cell_count_cutoff=5,
    cell_percentage_cutoff2=0.03,
    nonz_mean_cutoff=1.12
)

In [ ]:
ref = ref[:, selected].copy()
ref

Lets also remove mitochondrial and ribosomal genes from the main matrix as they're likely to be uninformative.

In [ ]:
mt = ref.var_names.str.startswith("mt-")
rb = ref.var_names.str.startswith("Rp")

ref.obsm["mt"] = ref[:, mt].X.toarray()
ref.obsm["rb"] = ref[:, rb].X.toarray()

ref = ref[:, ~(mt | rb)].copy()
ref

### 1.3 Estimation of reference cell type signatures

The signatures are estimated from scRNA-seq data, accounting for batch effect, using a Negative binomial regression model.

First, we prepare anndata object for the regression model.
Then, we train the model to estimate the reference cell type signatures.

Note that to achieve convergence on your data (i.e., to get stabilization of the loss) you may need to increase `max_epochs=250` (see below).

Note also that here we are using `batch_size=2500` which is much larger than [scvi-tools](https://scvi-tools.org/) default and performs training on all cells in the data (`train_size=1`) - both parameters are defaults.

It is much faster to use GPU for training so we will use "slurm magic" to submit this cell to the GPU slurm queue for processing

**NOTE:**
this step can take a while (approx. 10 min), especially if running on CPU (much longer).
Therefore, we have precomputed a model file for you here that you can load instead.
To do so, feel free to skip to section `1.4`.

In [ ]:
%%slurm_exec -i ref, -o mod --time=00:20:00 --partition=gpu --gpus=1 --cpus=2 --mem=20G
cell2location.models.RegressionModel.setup_anndata(
    adata=ref,
    batch_key='Sample',   # 10X reaction / sample / batch
    labels_key='CellType' # cell type, covariate used for constructing signatures
)
mod = cell2location.models.RegressionModel(ref)
mod.train(max_epochs=250)

Check for convergence:

In [ ]:
mod.plot_history(20)

Next, we extract relevant model statistics and store them in [AnnData](https://anndata.readthedocs.io/en/stable/) reference object.
The function samples the trained [cell2location](https://github.com/BayraktarLab/cell2location) model’s posterior and stores summary statistics (means/uncertainty) in the [AnnData](https://anndata.readthedocs.io/en/stable/).
This makes the results reusable for plotting and downstream analysis without the model.

As before, this step may take a while to run, so we have prepared a precomputed object.
To do so, **skip to step** `1.4`.

In [ ]:
%%slurm_exec -i ref,mod -o ref,mod --time=00:20:00 --partition=gpu --gpus=1 --cpus=2 --mem=20G
ref = mod.export_posterior(
    ref,
    sample_kwargs={
        'num_samples': 1000,
        'batch_size': 2500
    }
)

### 1.4 Loading and saving the model

We can save the trained model to re-use later.

In [ ]:
# Saving your trained model
mod.save(OUTPUT_FOLDERNAME, overwrite=True)

adata_file = f"{OUTPUT_FOLDERNAME}/ref.h5ad"
ref.write(adata_file)

In [ ]:
# Reloading your trained model
ref = sc.read_h5ad(os.path.join(OUTPUT_FOLDERNAME, 'ref.h5ad'))
mod = RegressionModel.load(
    OUTPUT_FOLDERNAME,
    ref
)

The code below loads a model we pre-trained earlier, so if you trained your own, **skip this step**.

In [ ]:
# Loading our precomputed model
ref = sc.read_h5ad(os.path.join(f"{PRECOMPUTED_FOLDERNAME}/cell2location/", 'ref.h5ad')) # TODO: file not found
mod = RegressionModel.load(
    f"{PRECOMPUTED_FOLDERNAME}/cell2location/regression",
    ref
)

### 1.5 Export estimated expression in each cluster

Next, we want to use our regression model to estimate average expression of each gene across all cell type labels in the reference. 

In [ ]:
if 'means_per_cluster_mu_fg' in ref.varm.keys():
    inf_aver = ref.varm['means_per_cluster_mu_fg'][[f'means_per_cluster_mu_fg_{i}'
                                                    for i in ref.uns['mod']['factor_names']]].copy()
else:
    inf_aver = ref.var[[f'means_per_cluster_mu_fg_{i}'
                        for i in ref.uns['mod']['factor_names']]].copy()
inf_aver.columns = ref.uns['mod']['factor_names']
inf_aver.iloc[0:5, 0:5]

To deconvolute cell types, we need to ensure that both the reference single-cell dataset and the spatial transcriptomics dataset use the same genes.
If different references or pipelines were used to process the raw data, these might not be 100% the same between datasets.
We also filtered out uninformative genes in the single-cell dataset.
Finally, we find shared genes and subset both the [AnnData](https://anndata.readthedocs.io/en/stable/) object and the reference signatures.

In [ ]:
intersect = np.intersect1d(adata.var_names, inf_aver.index)
adata = adata[:, intersect].copy()
inf_aver = inf_aver.loc[intersect, :].copy()
inf_aver

[cell2location](https://github.com/BayraktarLab/cell2location) works on counts, so we need to move the original raw counts matrix back to `adata.X`. 

In [ ]:
adata.obs['sample'] = 'slide1'
adata.X = adata.layers['counts'].copy()

### 1.6 Estimate cell abundance

Next, we configure the [AnnData](https://anndata.readthedocs.io/en/stable/) object and initialise a [cell2location](https://github.com/BayraktarLab/cell2location) model with sample-level batching and prior information on cell states.

We then train the model on all spatial spots to estimate cell-type abundances everywhere in the tissue.

As before, this step may take a while to run (approx. 30 min), so we have prepared a precomputed object.
To use it, **skip to step** `1.7`.

In [ ]:
%%slurm_exec -i inf_aver,adata -o mod --time=00:60:00 --partition=gpu --gpus=1 --cpus=2 --mem=20G

cell2location.models.Cell2location.setup_anndata(
    adata=adata,
    batch_key="sample"
)

mod = cell2location.models.Cell2location(
    adata,
    cell_state_df=inf_aver,
    N_cells_per_location=20,
    detection_alpha=20
)

mod.train(
    max_epochs=20000,
    batch_size=None,
    train_size=1
)

Next, we extract relevant model statistics and store them in anndata reference object. 

As before, this step may take a while to run (approx 1 min), so we have prepared a precomputed object.
To use it, **skip to step** `1.7`.

In [ ]:
%%slurm_exec -i adata,mod -o adata,mod --time=00:20:00 --partition=gpu --gpus=1 --cpus=2 --mem=20G

adata = mod.export_posterior(
    adata,
    sample_kwargs={
        'num_samples': 1000,
        'batch_size': mod.adata.n_obs
    }
)

### 1.7 Saving and loading model

As in previous step, you can save your own model and load a pre-computed model.

In [ ]:
mod.save(f"{OUTPUT_FOLDERNAME}/deconvolution", overwrite=False)
adata.write(f"{OUTPUT_FOLDERNAME}/day1_with_cell2location.h5ad")

If you ever need to reload your own model:

In [ ]:
adata = sc.read_h5ad(os.path.join(f"{OUTPUT_FOLDERNAME}/deconvolution/", 'day1_with_cell2location.h5ad'))
mod = cell2location.models.Cell2location.load(
    f"{OUTPUT_FOLDERNAME}/cell2location/deconvolution",
    adata
)

As earlier, if you trained your own, **skip this step**.

In [ ]:
adata = sc.read_h5ad(os.path.join(f"{PRECOMPUTED_FOLDERNAME}/cell2location/", 'adata_with_cell2location.h5ad'))
mod = cell2location.models.Cell2location.load(
    f"{PRECOMPUTED_FOLDERNAME}/cell2location/deconvolution",
    adata
)

In [ ]:
adata

### 1.8 Visualising cell type abundances

Now lets inspect the results.
We can take a look at the cell type estimate table for each spot here:

In [ ]:
adata.obsm['q05_cell_abundance_w_sf']

And we can also add cell type abundance estimates to the meta data for plotting

In [ ]:
adata.obs[adata.uns['mod']['factor_names']] = adata.obsm['q05_cell_abundance_w_sf']
adata.obs

We can now use the predicted abundances for plotting as before.

Note that in these plots, we clipped the upper abundance range to enhance the visual contrast a bit.

In [ ]:
sq.pl.spatial_scatter(
    adata,
    cmap='viridis',
    color=['T-Cells'],
    vmax=3
)
sq.pl.spatial_scatter(
    adata,
    cmap='viridis',
    color=['Fibroblasts'],
    vmax=3
)
sq.pl.spatial_scatter(
    adata,
    cmap='viridis',
    color=['Enterocytes'],
    vmax=3
)

We can also use [cell2location](https://github.com/BayraktarLab/cell2location)'s blended plot function to visualise co-localisation of multiple cell types at once, although this gets visually messy if attempting to look at more than one cell type.

Play around with different visualisations!

In [ ]:
clust_labels = ['T-Cells', 'Fibroblasts', 'Enterocytes']

clust_col = ['' + str(i) for i in clust_labels] 

with mpl.rc_context({'figure.figsize': (15, 15)}):
    fig = cell2location.plt.plot_spatial(
        adata=adata,
        color=clust_col, 
        labels=clust_labels,
        show_img=True,
        style='fast',
        max_color_quantile=0.9,
        circle_diameter=6,
        colorbar_position='right'
    )

We can visualise the predicted abundance of each cell type per cluster using violin plots, which can be a bit clearer.

For example, we can see that in our original clusters, T-Cells are highest in cluster 4. 

> **Exercise:**
How does this compare to our other clustering solutions with spatial approaches?

In [ ]:
sc.pl.violin(
    adata,
    keys="T-Cells",
    groupby="clusters",
    stripplot=False,
    jitter=False
)

We can also calculate (scaled) mean cell type abundance per cluster and visualise as heatmap.
This can be a quick way of understanding which cluster/region various cell types are most enriched in at a glance.

For example, we can see that in additional to T-Cells, cluster 4 also has high abundance of macrophages, B-cells and plasma cells, as would be expected from immune follicles.
Myofibroblasts and fibroblasts are also co-localising - we would expect this to be the case outside of epithelial regions, but it's also worth bearing in mind that these two cell types share a lot of gene expression programs, and it's possible co-localisation in these cases is an inaccuracy of deconvolution. Always be critical of your results!

In [ ]:
cell_types = adata.uns["mod"]["factor_names"]

ab = adata.obs[cell_types]

ab_cluster = ab.groupby(adata.obs["clusters"]).mean()
ab_cluster_z = ab_cluster.apply(
    zscore,
    axis=0
)
cmap = sns.diverging_palette(
    h_neg=220,
    h_pos=10,
    as_cmap=True
)

sns.clustermap(
    ab_cluster_z,
    cmap=cmap,
    center=0,
    linewidths=0,
    rasterized=True
)

We can also do a quick, basic cell type co-localisation check by simply checking the correlations between predicted cell type counts and plotting the correlation matrix as below.
We can see here that we have some correlations that make sense - for example, transit amplifying cells and stem cells, both which we know should be located at the base of the crypt, have a strong co-localisation.
Predicted B-cell localisation here is less biologically interpretable, which hints that our cell type deconvolution may not have worked accurately in all cases.

You should always examine these results skeptically!
Cell type deconvolution is not a "solved" problem in spatial transcriptomics, and methods that work well for certain tissues can struggle in others. 

In [ ]:
corr = adata.obs[cell_types].corr()
cmap = sns.diverging_palette(
    h_neg=220,
    h_pos=10,
    as_cmap=True
)
sns.clustermap(
    corr,
    cmap=cmap,
    center=0,
    square=True
)

Lets save the [anndata](https://anndata.readthedocs.io/en/stable/) with cell type predictions so we can easily load it later.

In [ ]:
adata_file = f"{OUTPUT_FOLDERNAME}/adata_with_cell2location.h5ad"
adata.write(adata_file)

## 2. Spatial Neighbours

Correlation analyses allow us to look at the signatures in the same spots, but what if we want to ask more precise questions about tissue areas that are adjacent to each other?
We can do quite a bit of with spatial neighbour analyses. 

Analogously to a nearest neighbour graph in single-cell analysis, where neighbours are based on transcriptional similarity to each other, we can construct a spatial nearest neighbour graph, where neighbourhood is simply defined by physical space.
We can then use this graph to look at co-localisation. 

We can construct this using [squidpy](https://squidpy.readthedocs.io/en/stable/) `spatial_neighbors()` function.

In [ ]:
sq.gr.spatial_neighbors(adata)
adata

[squidpy](https://squidpy.readthedocs.io/en/stable/) has several in-built functions for further analysis.

For example, we might want to know which of our clusters are in close physical proximity.
We can now use the neighbourhood enrichment test to visualise this.
We can see that this identifies distinct domains, where clusters 1 and 10 co-localise in the outer "roll" of the tissue, while 0 and 3 in the middle layer. 

In [ ]:
sq.gr.nhood_enrichment(
    adata,
    cluster_key="clusters"
)
sq.pl.nhood_enrichment(
    adata,
    cluster_key="clusters",
    method="average",
    figsize=(5, 5)
)
sq.pl.spatial_scatter(
    adata,
    color="clusters"
)

Previous cluster co-occurence analysis tested whether there is enrichment in the local neighbourhood.

We can extend this question further and ask at what distance do clusters have strongest co-localisation.
For this, we can calculate a co-occurance score across increasing distances.

For example here if we select cluster 0 as a "viewpoint", we can see that the co-occurence with cluster 3 is strongest at shorter distances and then decreases.
Co-occurence with cluster 2 is lower at low distances, but peaks at mid distances. 

Note that the distances here will always be distances calculated from whatever coordinate system your data is in.
This could be pixels, or more meaningful microns - e.g. in the case of Xenium data. 

In [ ]:
sq.gr.co_occurrence(
    adata,
    cluster_key="clusters"
)
sq.pl.co_occurrence(
    adata,
    cluster_key="clusters",
    clusters="0",
    figsize=(8, 5)
)

In spatial data, we can also do distance-based analyses.
[Squidpy](https://squidpy.readthedocs.io/en/stable/) has a helper function that will allow us to calculate distances between spots and any selected feature. 

Here, for example, let's try to calculate distance of all spots to the nearest cluster 4 spot. 

In [ ]:
sq.tl.var_by_distance(
    adata=adata,
    groups="4",
    cluster_key="clusters",
    design_matrix_key="distance_to_cluster_4"
)

The results table is stored here:

In [ ]:
adata.obsm["distance_to_cluster_4"]

Lets add the distance as a column to obs for easier plotting and visualise the calculated distances.
We can see that that we get a nice, smooth "halo" of spot distances to the nearest cluster 4 spot/follicle.

In [ ]:
adata.obs["dist_cl4"] = adata.obsm["distance_to_cluster_4"]["4_raw"]
sq.pl.spatial_scatter(adata, color="dist_cl4")

Now that we have calculated a distance of interest, we can also visualise how gene expression varies across that distance.

For example, here we can see that Cd74 gene is highly expressed close to cluster 4 spots and decreases the further we go away from it.

In [ ]:
sq.pl.var_by_distance(
    adata=adata,
    design_matrix_key="distance_to_cluster_4",
    var=["Cd74"],
    anchor_key="4",
    show_scatter=False
)

For more specific analyses, we might want to select spots based on their adjacently to other features.
For example, we might be interested in whether the spots immediatly surrounding lymphoid structures express any interesting genes. 

To answer a question like that, we need to be able to identify and select spots based on their proximity to other spots.
We can use the distances we calculated earlier by thresholding spots based on distance - here, since cluster 4 were our spots of interest, the distance to nearest cluster 4 spot is zero, so we assign those as our "Spots of Interest" group, and then threshold other spots based on a small distance that will pick up only the first "layer" of adjacent spots. 

Then, we can make a new variable in the meta data of our anndata object with our groupings and visualise it:

In [ ]:
# Initialise the new column
adata.obs["spot_group"] = "Other Spots"
# Identify the spots of interest
adata.obs.loc[adata.obs["dist_cl4"] == 0, "spot_group"] = "Spots of Interest"
# Identify spots adjacents to the spots of interest
adata.obs.loc[(adata.obs["dist_cl4"] > 0) & (adata.obs["dist_cl4"] <= 30), "spot_group"] = "Adjacent Spots"
# Convert to categorical data type
adata.obs["spot_group"] = adata.obs["spot_group"].astype("category")
# Plot
sq.pl.spatial_scatter(adata, color="spot_group")

Now we can run marker detection between groups to identify genes of interest.

Remember: we replaced our previous matrix with raw counts for cell type deconvolution analysis and so we should normalise our data again. 

**NOTE:** we also filtered out a whole bunch of uninformative low expression genes, so if you're interested in these, you should reload an earlier [AnnData](https://anndata.readthedocs.io/en/stable/) object for this instead!

In [ ]:
sc.pp.normalize_total(
    adata,
    inplace=True
)
sc.pp.log1p(adata)

Now, we can run a marker search between selected and adjacent tissue spots. 

In [ ]:
adata.obs["spot_group"]

In [ ]:
sc.tl.rank_genes_groups(
    adata,
    groupby="spot_group",
    method="wilcoxon",
    use_raw=False,
    groups=["Adjacent Spots"],
    reference="Spots of Interest"
)
sc.get.rank_genes_groups_df(
    adata,
    group="Adjacent Spots"
)

For example, we can see that *Cd74* is higher within the follicles (spots of interest), while *Atp1a1* is lower. 

We can also compare cell type deconvolution results.
Again, we can see that T-cells are more abundant in our spots of interest, but not in the layer immediately outside.

You could also compare adjacent spots with all other spots. 

In [ ]:
sc.pl.violin(
    adata,
    keys=["Cd74", "Atp1a1"],
    groupby="spot_group",
    stripplot=False,
    jitter=False
)

In [ ]:
sc.pl.violin(
    adata,
    keys=["T-Cells", "Macrophages"],
    groupby="spot_group",
    stripplot=False,
    jitter=False
)

## 3. Working with multiple samples

Typically, you will not be working with just one sample but multiple ones across different conditions.
We will next discuss how to work with multiple slices. 

Here, we have prepared a second [AnnData](https://anndata.readthedocs.io/en/stable/) object from "DAY14 DSS" treated mouse intestine, which has been pre-processed through the exact same steps as the DAY0 healthy tissue section that we used during day 1.

**NOTE:** You can also do this yourself, by going through yesterday's workflow using the [spaceranger](https://www.10xgenomics.com/support/software/space-ranger/latest) outputs for this sample available here:

```
/project/shared/r/3_r_spatial/DATA/VISIUM_V1_MOUSE_INTESTINE/spaceranger/SRR14083627_DSS_DAY14
```

In [ ]:
section1 = sc.read_h5ad(os.path.join(PRECOMPUTED_FOLDERNAME, 'day1.h5ad'))
section2 = sc.read_h5ad(os.path.join(PRECOMPUTED_FOLDERNAME, 'day14_precomputed.h5ad'))

Let's inspect both tissue sections.
We have identified clusters on both slides, but they are individual section clusters and therefore not the same between slides.
For downstream analysis, we need a unified grouping, such that cluster 1 in control slide is the same as cluster 1 in treatment section if we want to do any comparisons. 

In [ ]:
sq.pl.spatial_scatter(
    section1,
    color="clusters"
)
sq.pl.spatial_scatter(
    section2,
    color="clusters"
)

#### Merging datsets

The `concat()` function in [scanpy](https://scanpy.readthedocs.io/en/stable/) is used to combine multiple objects into a single object. 

This is particularly useful when you have data from different sections, conditions or batches that you want to analyse together. 

In this case, the code merges two [AnnData](https://scanpy.readthedocs.io/en/stable/) objects (e.g., from different tissue sections) into one combined [AnnData](https://scanpy.readthedocs.io/en/stable/) object.

This should also merge your `spatial` and `image` slots such that each image is retained and accessible for plotting.

**TIP:**
Basic meta data has been added to individual section objects already.
To keep the meta data tidy, ensure the meta data columns across all objects to be merged are the same.

In [ ]:
merged = sc.concat(
    [
        section1,
        section2
    ],
    label="dataset",
    uns_merge="unique",
    keys=[
        'Day0',
        'Day14'
    ],
    index_unique="-",
)

In [ ]:
merged

In [ ]:
merged.obs

We can do a sanity check to see how many spots there are in the object in each tissue section.

In [ ]:
merged.obs['dataset'].value_counts()

To analyse merge object data, generally we want to repeat the basic workflow we carried out for one sample in day 1, except now do this for the merged object.
As the data were previously normalised, we don't need to do those steps again.

**We want to start from variable gene selection.**
One key difference is that when we calculate highly variable genes for multiple samples, it is important that gene selection is performed in a *batch-aware* way.
This is because genes that are variable across the whole dataset could be capturing batch effects rather than the biological signals we are interested in. 

We can perform batch-aware highly variable gene selection by setting the `batch_key=` argument in the [scanpy](https://scanpy.readthedocs.io/en/stable/) `highly_variable_genes()` function.
[scanpy](https://scanpy.readthedocs.io/en/stable/) will then calculate HVGs for each batch separately and combine the results by selecting those genes that are highly variable in the highest number of batches. 

We use the [scanpy](https://scanpy.readthedocs.io/en/stable/) function here because it has this batch awareness built in.
For other methods, we would have to run them on each batch individually and then manually combine the results.

In [ ]:
sc.pp.highly_variable_genes(
    merged,
    n_top_genes=2000,
    flavor="seurat",
    batch_key='dataset'
)
merged.var

Next, we repeat the PCA and clustering analysis as before

In [ ]:
sc.pp.pca(merged)
sc.pp.neighbors(
    merged,
    n_pcs=10
)
sc.tl.umap(merged)
sc.tl.leiden(
    merged,
    key_added="cluster",
    flavor="igraph",
    n_iterations=2,
    resolution=0.5
)

We can visualise the results over the UMAP embedding.
However, when we check the clusters, we can see that spots from different slides form their own clusters.
We don't want this, but rather to find joint regions!

In [ ]:
sc.pl.umap(
    merged,
    color=['dataset', 'cluster']
)

In [ ]:
sq.pl.spatial_scatter(
    merged,
    color="cluster",
    library_key="dataset",
    library_id=["Day0", "Day14"]
)

[harmony](https://github.com/immunogenomics/harmony) is an algorithm designed for integrating single-cell data from different batches or conditions while preserving biological variability.
It is particularly effective in handling batch effects and other sources of technical variation.
We can generally use most methods designed for single-cell analysis on spatial transcriptomics data integration - e.g. another good alternative is [scVI](https://scvi-tools.org/). 

[harmony](https://github.com/immunogenomics/harmony) works by correcting reduced dimensions between datasets.
In most use cases, this corrects the standard PCA matrix derived from gene expression.
However, we can use [harmony](https://github.com/immunogenomics/harmony) on any dimensionality reduction - this means that, for example, you could substitute this with the features/PCA calculated using [Banksy](https://github.com/prabhakarlab/Banksy) algorithm to use spatial clustering with [harmony](https://github.com/immunogenomics/harmony) integration together.

In [ ]:
sc.external.pp.harmony_integrate(
    merged,
    key='dataset',
    basis='X_pca',
    adjusted_basis='X_pca_harmony'
)

Once we have harmonised PCs, we can now use these as input for clustering and umap instead of PCA.

In [ ]:
sc.pp.neighbors(
    merged,
    n_pcs=10,
    use_rep="X_pca_harmony"
)
sc.tl.umap(merged)
sc.tl.leiden(
    merged,
    resolution=0.5,
    key_added="harmony_cluster"
)

We can see that we now have clusters where the two tissue sections are aligned. 

In [ ]:
sc.pl.umap(
    merged,
    color=['dataset','harmony_cluster']
)

We can now see that harmonised clusters appear to be consistent between two tissue sections.

In [ ]:
sq.pl.spatial_scatter(
    merged,
    color="harmony_cluster",
    library_key="dataset",
    library_id=["Day0", "Day14"]
)

Let's have a quick look to see if we can find some differences between datasets.
Since we only have two sections, we can still use wilcoxon test to compare them. 

In [ ]:
sc.tl.rank_genes_groups(
    merged,
    groupby="dataset",
    method="wilcoxon",
    use_raw=False
)

In [ ]:
sc.get.rank_genes_groups_df(
    merged,
    group="Day0"
)

As an example, can see one differentially expressed gene, *Krt13*, has been detected in the DSS-treated tissue sample but not in the control.
We did a simple whole slide level comparison, but we could also modify the above to ask more specific questions about differences within certain common regions. 

In [ ]:
sq.pl.spatial_scatter(
    merged,
    color="Krt13",
    library_key="dataset",
    library_id=["Day0", "Day14"],
    vmin=0,
    vmax=merged[:, "Krt13"].X.max()
)

Let's save the merged anndata object at the end of this analysis.

In [ ]:
adata.write(os.path.join(OUTPUT_FOLDERNAME, f'day2_merged.h5ad'))

## 4. Working with HD data at multiple resolutions

### Higher resolution datsets - Visium HD

Finally, we will briefly explore how higher resolution sequencing-based datasets compare to more common spot-based datasets. 

Generally, due to increased resolution, this analysis takes much, **much**, longer and requires more computational resources. 

Here, we will use a mouse intestine [Visium HD](https://www.10xgenomics.com/platforms/visium/product-family) dataset, which will enable us to compare (at least qualitatively) the differences between two technologies as the tissues are very similar. 

First, we want to load the data.

Previously, we have used [squidpy](https://squidpy.readthedocs.io/en/stable/) to create [AnnData](https://anndata.readthedocs.io/en/stable/) objects from our spatial data.
Here, we will use [SpatialData](https://spatialdata.scverse.org/en/stable/) data structure instead, as [squidpy](https://squidpy.readthedocs.io/en/stable/) does not support the new HD data formats. 

You can check the [SpatialData](https://spatialdata.scverse.org/en/stable/) documentation for additional information on the data structures: 

https://spatialdata.scverse.org/en/latest/tutorials/notebooks/notebooks/examples/intro.html

In [ ]:
path = f"{DATA_FOLDERNAME}/visium_hd_mouse_intestine"
hd = visium_hd(path, dataset_id='Visium_HD_Mouse_Small_Intestine')

If we inspect the data object `hd`, you can see that we have ingested three main parts:

**images**,
which contain various resolutions of our H&E images

**shapes**,
which in other spatial datasets would contain cell segmentation polygons (but here we keep spatial bins)

and finally there are three **tables**.

The tables are stored as [AnnData](https://anndata.readthedocs.io/en/stable/) objects and are analogous to [squidpy](https://squidpy.readthedocs.io/en/stable/).

We have three tables, where bins have been summarised at different resolutions.
As you can hopefully appreciate, the highest resolution bin ingests more than 5 million data points, which could be very computationally intense to work with!

In [ ]:
hd

We can access individual anndata objects and operate on them as before with standard scanpy/squidpy functions and workflows. For example, we can inspect the 16um bin anndata object here:

In [ ]:
hd['square_016um']

For the purposes of this tutorial, we will just carry out some very basic filtering on this dataset to remove spots with fewer than 200 counts detected.

In an ideal workflow, you would explore the QC metrics as we did in day 1.  

In [ ]:
sc.pp.filter_cells(
    hd['square_016um'],
    min_counts=200,
    inplace=True
)
hd['square_016um']

Next, lets run some quick normalisation, PCA and clustering analysis

In [ ]:
sc.pp.normalize_total(
    hd['square_016um'],
    inplace=True
)
sc.pp.log1p(
    hd['square_016um']
)
sc.pp.highly_variable_genes(
    hd['square_016um'],
    flavor="seurat",
    n_top_genes=2000
)
sc.pp.pca(
    hd['square_016um']
)
sc.pp.neighbors(
    hd['square_016um'],
    n_pcs=20
)
sc.tl.umap(
    hd['square_016um']
)
sc.tl.leiden(
    hd['square_016um'],
    key_added="clusters",
    flavor="igraph",
    n_iterations=2,
    resolution=0.5
)

We can visualise the results with `umap()`.

In [ ]:
sc.pl.umap(
    hd['square_016um'],
    color=["clusters"],
    wspace=0.4
)

In [ ]:
hd['square_016um']

And we can use spatial scatter from [squidpy](https://squidpy.readthedocs.io/en/stable/) to visualise clusters.
Already, you can hopefully see that even that lowest resolution of bins gives us a much more detailed view of the mouse intestine than Visium V1 data in this swiss roll. 

In [ ]:
sq.pl.spatial_scatter(
    hd['square_016um'],
    shape=None,
    color="clusters"
)

Now lets repeat the analysis on higher resolution bins.
Again, we will do some very basic filtering to remove some spots, but in practice you would want to explore all QC metrics here. 

In [ ]:
sc.pp.filter_cells(
    hd['square_008um'],
    min_counts=200,
    inplace=True
)

In [ ]:
sc.pp.normalize_total(
    hd['square_008um'],
    inplace=True
)
sc.pp.log1p(
    hd['square_008um']
)
sc.pp.highly_variable_genes(
    hd['square_008um'],
    flavor="seurat",
    n_top_genes=2000
)
sc.pp.pca(
    hd['square_008um']
)
sc.pp.neighbors(
    hd['square_008um'],
    n_pcs=20
)
sc.tl.umap(
    hd['square_008um']
)
sc.tl.leiden(
    hd['square_008um'],
    key_added="clusters",
    flavor="igraph",
    n_iterations=2,
    resolution=0.5
)

In [ ]:
sc.pl.umap(
    hd['square_008um'],
    color=["clusters"],
    wspace=0.4
)

Here we can see that we get a lot more structure and detail in the data.

In [ ]:
sq.pl.spatial_scatter(
    hd['square_008um'],
    shape=None,
    color="clusters"
)

We can define a crop to zoom into a smaller region of the slide to compare clusters between resolutions in a bit more detailed.

In [ ]:
crop = (12000, 12000, 16000, 16000)
sq.pl.spatial_scatter(
    hd['square_016um'],
    shape=None,
    color="clusters",
    crop_coord=crop,
    size=30
)
sq.pl.spatial_scatter(
    hd['square_008um'],
    shape=None,
    color="clusters",
    crop_coord=crop,
    size=3
)

We can save spatial data objects to disk as [zarr](https://zarr.dev/) files to reload at a later time point. 

In [ ]:
hd.write(os.path.join(OUTPUT_FOLDERNAME, "visium_hd.zarr"))

### Single Cell Segmentation of HD Data 

The above analysis uses bins as they are measured on the slide for analysis.
But they generally do not correspond to cells.
In the workflow below, we will go through the steps we can take to partition the data into cells instead.

We will use a method called [bin2cell](https://github.com/Teichlab/bin2cell), which wraps [stardist](https://github.com/stardist/stardist) algorithm for cell and nuclei segmentation and provides functions for intersecting the outputs with spatial transcriptomics data.

You can read more about the method here:

https://academic.oup.com/bioinformatics/article/40/9/btae546/7754061

More specifically on stardist:

https://github.com/stardist/stardist

NOTE: Latest version of [spaceranger](https://www.10xgenomics.com/support/software/space-ranger/latest) now implements are similar approach that we take here.
In the data that we are using for this lesson, this was not yet available.
In our approach, we will use a combination of nuclei-based segmentation and transcript density based segmentation, while [spaceranger](https://www.10xgenomics.com/support/software/space-ranger/latest) uses the nuclei only. 

There are multiple other high resolution spatial platforms that generate very high resolution bin data (e.g. stereo-seq) where cell level segmentation may not be generated automatically and you might need to apply this approach yourself.

In [ ]:
path = f"{DATA_FOLDERNAME}/visium_hd_mouse_intestine/binned_outputs/square_002um/"
source_image_path = f"{DATA_FOLDERNAME}/visium_hd_mouse_intestine/image.tiff"
spaceranger_image_path = f"{DATA_FOLDERNAME}/visium_hd_mouse_intestine/spatial/"

In [ ]:
adata = b2c.read_visium(
    path,
    source_image_path = source_image_path,
    spaceranger_image_path = spaceranger_image_path
)
adata.var_names_make_unique()
adata

In [ ]:
sc.pp.calculate_qc_metrics(
    adata,
    inplace=True
)

In [ ]:
sc.pp.filter_cells(
    adata,
    min_counts=1,
    inplace=True
)

In [ ]:
os.makedirs(os.path.join(OUTPUT_FOLDERNAME, "stardist"), exist_ok=True)

First we want to rescale the high resolution H&E image from visium to roughly the resolution that [stardist](https://github.com/stardist/stardist) nuclei detection model will have been trained on.
We can use the scale function provided by [bin2cell](https://github.com/Teichlab/bin2cell).

In [ ]:
mpp = 0.3
b2c.scaled_he_image(
    adata,
    mpp=mpp,
    save_path=os.path.join(OUTPUT_FOLDERNAME, "stardist", "he.tiff")
)

Next, because we are working with the smallest 2um bins that are measured on the slide, we (optionally) want to correct for any artefacts from uneven capture between bins, as the smallest bins are in practice slightly uneven, which causes small "stripe" artefacts in the data.
This is generally not an issue in larger size bins we have been working with previously, as signal is aggregated across a larger area. 

In [ ]:
b2c.destripe(
    adata,
    counts_key="total_counts"
)

In [ ]:
adata

We then run [stardist](https://github.com/stardist/stardist), which detects nuclei in our H&E images.
The below step might take a bit of time (~15 mins) to run, therefore **you can skip this step** and use our pre-computed segmentation in the next step. 

Lowering `prob_thresh=` to make the calls less stringent - you might want to tweak this to "rescue" more cells/nuclei, but it will also create more objects which might not be real cells. 

`nms_thresh=` parameter tells the segmentation what's the expected overlap between objects.
Tweaking this can help resolve segmentation in areas which are densely packed, but can also over-partition a one cell in multiple objects.

Expect the cell below to take around 12 min to run.

In [ ]:
%%slurm_exec --time=00:20:00 --partition=gpu --gpus=1 --cpus=2 --mem=30G

b2c.stardist(
    image_path=os.path.join(OUTPUT_FOLDERNAME, "stardist", "he.tiff"),
    labels_npz_path=os.path.join(OUTPUT_FOLDERNAME, "stardist", "he.npz"),
    stardist_model="2D_versatile_he",
    prob_thresh=0.01,
    nms_thresh=0.5
)

Once nuclei detection is complete, we can read in the results and attach the segmentations to our anndata object. This step assigns individual bins to nuclei boundaries. 

**NOTE:**
If you'd like to load pre-computed segmentation, the equivalent files are stored here:

```
/project/shared/python/5_python_spatial_omics/data/precomputed/day1/stardist
```

In [ ]:
b2c.insert_labels(
    adata,
    labels_npz_path=os.path.join(OUTPUT_FOLDERNAME, "stardist", "he.npz"),
    basis="spatial",
    spatial_key="spatial_cropped_150_buffer",
    mpp=mpp,
    labels_key="labels_he"
)

Cells are not just nuclei.
One approach we can take here is to expand the nuclei boundaries to include additional nearby spots that would be expected to fall in the cytoplasm.
This will not be 100% accurate, as ideally you'd need to have a specific cell boundary image stain to infer the cell boundaries with more precision, but it's a good approach when we only have H&E images.

`algorithm="volume_ratio"` assigns each nucleus a custom expansion distance based on its bin count based on the expected ratio between cell and nucleus volume.
If a bin is equidistant between two nuclei, its a assigned to a nucleus based on gene expression profile similarity.

Here, you could alternatively set a fixed bin expansion instead.

In [ ]:
b2c.expand_labels(
    adata,
    labels_key='labels_he',
    expanded_labels_key="labels_he_expanded",
    algorithm="volume_ratio"
)

The images are very large and so it can be hard to visualise how individual spots are assigned to cells.
Here, we pick a very small field of view to get a quick visual of how nuclei segmentation is working.  

[bin2cell](https://github.com/Teichlab/bin2cell) bin labels are integers, where 0 means a bin is not assigned to any cell.
Therefore, we also want to remove any non-cell bins for visual clarity.

In [ ]:
mask = (
    (adata.obs['array_row'] >= 1530) &
    (adata.obs['array_row'] <= 1550) &
    (adata.obs['array_col'] >= 390) &
    (adata.obs['array_col'] <= 410)
)

adata_subset = adata[mask].copy()

adata_subset = adata_subset[adata_subset.obs['labels_he']>0].copy()

adata_subset.obs['labels_he'] = adata_subset.obs['labels_he'].astype('category')

sc.pl.spatial(
    adata_subset,
    color="labels_he",
    img_key="0.3_mpp_150_buffer",
    basis="spatial_cropped_150_buffer",
    palette="tab20"
)

Nuclei detection can be unreliable, and we might have cells where nuclei are not picked up so well either by staining or segmentation.
[bin2cell](https://github.com/Teichlab/bin2cell) allows us to (optionally) use gene expression density-based cell detection step on top of nuclei segmentation. 

To do this, we first need to create a gene expression intensity image from the bins.
Here, we use adjusted, "destriped" UMI counts to obtain a more uniform signal. 

In [ ]:
b2c.grid_image(
    adata,
    "n_counts_adjusted",
    mpp=mpp,
    sigma=5,
    log1p=True,
    save_path=os.path.join(OUTPUT_FOLDERNAME, "stardist", "gex.tiff")
)

Then, as before, we can call [stardist](https://github.com/stardist/stardist), but this time instead of segmenting the image on H&E stain, we use the transcript density map instead.

Expect the cell below to take around 3 min to run.

In [ ]:
%%slurm_exec --time=00:20:00 --partition=gpu --gpus=1 --cpus=2 --mem=20G

b2c.stardist(
    image_path=os.path.join(OUTPUT_FOLDERNAME, "stardist", "gex.tiff"),
    labels_npz_path=os.path.join(OUTPUT_FOLDERNAME, "stardist", "gex.npz"),
    stardist_model="2D_versatile_fluo",
    prob_thresh=0.05,
    nms_thresh=0.5
)

Read in the segmentations and add them to our [AnnData](https://anndata.readthedocs.io/en/stable/) object.

**Note:**
If you'd like to load pre-computed segmentation, the equivalent files are stored here:

```
/project/shared/python/5_python_spatial_omics/data/precomputed/day1/stardist
```

In [ ]:
b2c.insert_labels(
    adata,
    labels_npz_path=os.path.join(OUTPUT_FOLDERNAME, "stardist", "gex.npz"),
    basis="array",
    mpp=mpp,
    labels_key="labels_gex"
)

Lets visualise in the same spatial region as our nuclei-based segmentation

In [ ]:
adata_subset = adata[mask].copy()

adata_subset = adata_subset[adata_subset.obs['labels_gex']>0].copy()

adata_subset.obs['labels_gex'] = adata_subset.obs['labels_gex'].astype('category')

sc.pl.spatial(
    adata_subset,
    color="labels_gex",
    img_key="0.3_mpp_150_buffer",
    basis="spatial_cropped_150_buffer",
    palette="tab20"
)

We can also visualise actual cell boundary polygons:

In [ ]:
crop = b2c.get_crop(
    adata_subset,
    basis="array",
    mpp=mpp
)

rendered = b2c.view_labels(
    image_path=os.path.join(OUTPUT_FOLDERNAME, "stardist", "gex.tiff"),
    labels_npz_path=os.path.join(OUTPUT_FOLDERNAME, "stardist", "gex.npz"),
    crop=crop,
    stardist_normalize=True
)
plt.imshow(rendered)

For downstream data analysis, we can combine the nuclei based segmentation with UMI-based segmentation results to get joint labels.

In [ ]:
b2c.salvage_secondary_labels(
    adata,
    primary_label="labels_he_expanded",
    secondary_label="labels_gex",
    labels_key="labels_joint"
)

At this point, we can generate a new [AnnData](https://anndata.readthedocs.io/en/stable/) object which aggregates the UMI counts per cell.
We can see that we have a lot of cells!

In [ ]:
adata_cell = b2c.bin_to_cell(
    adata,
    labels_key="labels_joint",
    spatial_keys=[
        "spatial",
        "spatial_cropped_150_buffer"
    ]
)
adata_cell

In [ ]:
adata_cell.var["mt"] = adata_cell.var_names.str.startswith("mt-")
adata_cell.var["rb"] = adata_cell.var_names.str.startswith("Rp")
sc.pp.calculate_qc_metrics(
    adata_cell,
    qc_vars=[
        "mt",
        "rb"
    ],
    inplace=True
)
adata_cell

In [ ]:
sns.histplot(
    adata_cell.obs,
    x="bin_count",
    kde=True,
    bins=60
)

Now, we can QC, cluster and analyse the data in the same way as before. Here again we apply a very quick filter to remove cells with low transcript counts and with very few or very large number of bins assigned to a cell - in practice, you should explore various QC thresholds to find a more optimal strategy. 

!!! Our filter is probably overly stringent, in that we remove a lot of cells from the analysis. This will help with the speed of this tutorial, but in practice you probably want to keep more cells. 

In [ ]:
mask = (adata_cell.obs["bin_count"] > 3) & (adata_cell.obs["total_counts"] > 200) & (adata_cell.obs["bin_count"] < 25)

print(f"Barcodes before filtering: {adata_cell.n_obs}")

adata_cell = adata_cell[mask].copy()

print(f"Barcodes after cell count filter: {adata_cell.n_obs}")

Next, we normalise and cluster the data as usual. As we have quite a large number of cells, this step can take some time.

In [ ]:
sc.pp.normalize_total(
    adata_cell,
    inplace=True
)
sc.pp.log1p(
    adata_cell
)
sc.pp.highly_variable_genes(
    adata_cell,
    flavor="seurat",
    n_top_genes=5000
)
sc.pp.pca(
    adata_cell
)
sc.pp.neighbors(
    adata_cell,
    n_pcs=20
)
sc.tl.umap(
    adata_cell
)
sc.tl.leiden(
    adata_cell,
    key_added="clusters",
    flavor="igraph",
    n_iterations=2,
    resolution=0.5
)

Now, we can visualise the data as before. The dataset is sparser than the 8um bins and depending on your biological question, bin level aggregation may be more suitable. 

How does the data look like if we use only nuclei-segmentation or just expression density segmentation results?

In [ ]:
sc.pl.umap(
    adata_cell,
    color=["clusters"],
    wspace=0.4
)

In [ ]:
sq.pl.spatial_scatter(adata_cell, shape=None, color="clusters")

In [ ]:
sc.pl.umap(adata_cell, color=["Myh11", "Cd74", "Epcam"], vmax="p99")

Save cell segmented [AnnData](https://anndata.readthedocs.io/en/stable/) object. 

In [ ]:
adata.write(os.path.join(OUTPUT_FOLDERNAME, f'day2_hd_cell_segmented.h5ad'))